In [1]:
!pip install yfinance
!pip install yahooquery
!pip install tvdatafeed
!pip install tensorflow
!pip install --upgrade mplfinance
from tvDatafeed import TvDatafeed, Interval
from yahooquery import Screener
import yfinance as yf   
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import random 
from tensorflow.keras.models import load_model
import os
import shutil
import mplfinance as mpl 
from datetime import datetime
import glob
from PIL import Image
import cv2
import gc
import gc
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('agg')
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Activation,Dropout,Flatten,Dense,AveragePooling2D,GlobalAveragePooling2D
!pip install -U git+https://github.com/twopirllc/pandas-ta
import pandas_ta as pta


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.4 MB 30.6 MB/s 
     |████████████████████████████████| 62 kB 1.8 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.28.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |██████████

In [2]:

def work_with_dir():
  if os.path.exists("/content/data/"):
    shutil.rmtree("/content/data/", ignore_errors=True)
    print("Data Folder Removed")
    os.mkdir("/content/data/")
  if not os.path.exists("/content/data/"):
    os.mkdir("/content/data/")
  if not os.path.exists("/content/extracted/"):
    os.mkdir("/content/extracted/")
  if not os.path.exists("/content/checkpoints/"):
    os.mkdir("/content/checkpoints/")


def get_crypto_syms():
   # 'all_cryptocurrencies_au','all_cryptocurrencies_ca','all_cryptocurrencies_eu','all_cryptocurrencies_gb','all_cryptocurrencies_in',
   screens = [
       'all_cryptocurrencies_us', 'all_cryptocurrencies_au', 'all_cryptocurrencies_ca', 'all_cryptocurrencies_eu', 'all_cryptocurrencies_gb', 'all_cryptocurrencies_in', ]
   s = Screener()
   symbols = []
   for i in screens:
      data = s.get_screeners(i, count=250)
      dicts = data[i]['quotes']
      syms = [d['symbol'] for d in dicts]
      for sym in syms:
        symbols.append(sym)
   # print(len(symbols))
   # pieces = 15
   # new_arrays = np.array_split(symbols, pieces)
   return symbols


def download_data(symbols, periodd, intervall):
  indexx = 100
  work_with_dir()
  for symbol in symbols:
    if ((symbols.index(symbol)+1) % 100 == 0):
      print(f" -- {indexx}", end="")
      indexx = indexx + 100
    try:
        data = yf.download(symbol, period=periodd,
                           interval=intervall, progress=False, show_errors=False)
        if data.empty:
           pass
        else:
            data.to_csv(f"/content/data/{symbol}.csv")
    except:
       print("Error!")
  print(" ")


def extract_data(how_many_future_candles, how_many_past_candles, each_row_past):
  print(f"Files In Data : {len(os.listdir('/content/data/'))}")
  pd.options.mode.chained_assignment = None
  files = os.listdir("/content/data/")
  print("Processing File:")
  now = datetime.now().strftime("%H%M%S")
  os.mkdir(f"/content/extracted/{now}/")
  index = 1
  for file in files:
     print(f"File Number {index}:", end=" ")
     each_file_proc(file, now, how_many_future_candles,
                    how_many_past_candles, each_row_past)
     index = index + 1
  print(" ")
  return now


def each_file_proc(file, now, how_many_future_candles, how_many_past_candles, each_row_past):

    address = f"/content/data/{file}"
    data = pd.read_csv(address)
    data = data.dropna()
    dfs = []
    suggs = []
    if len(data.columns) == 7:
      data = data.iloc[:, 1:]
    data["6_rsi"] =  pta.rsi(data['Close'], length = 6)
    data["14_rsi"] = pta.rsi(data['Close'], length = 14)
    data["26_rsi"] = pta.rsi(data['Close'], length = 26)
    data["52_rsi"] = pta.rsi(data['Close'], length = 52)

    # drop Adj Close and Volume
    data.drop(columns=['Adj Close', 'Volume'], inplace=True)
    data = np.array(data)
    data = data.astype(float)
    for i in range(each_row_past, data.shape[0]-which_future_or_past):

        rows = data[i-each_row_past:i, :]
        past_candles = []
        for z in range(1, how_many_past_candles+1):
          past_candles.append((data[i-z][3]+data[i-z][0])/2)
        past_candles = sum(past_candles)/len(past_candles)
        next_candles = []
        for z in range(0, how_many_future_candles):
          next_candles.append((data[i+z][3]+data[i+z][0])/2)
        next_candles = sum(next_candles)/len(next_candles)
        if next_candles > past_candles:
          suggs.append(1)
        else:
          suggs.append(0)

        x = pd.DataFrame(rows, columns=["Open", "High", "Low", "Close","6_rsi", "14_rsi", "26_rsi","52_rsi"])
        del rows
        del next_candles
        dfs.append(x)
        del x
    del data

    index = 0
    all_n = np.array(dfs).shape[0]
    for df in dfs:
        date_list = []
        rrng = np.array(df).shape[0]
        for i in range(rrng):
            date_list.append(datetime.fromordinal(
                datetime.today().toordinal()+i).strftime('%Y-%m-%d'))
        df.index = date_list
        df.index.name = "Date"
        df.index = pd.to_datetime(df.index)
        right_now = datetime.now().strftime("%H%M%S%f")
        address = f"/content/extracted/{now}/{right_now}_{suggs[index]}.png"
        
        #apds = [ 
        # mpl.make_addplot(df["6_rsi"],color='b',width=1),
        # mpl.make_addplot(df["14_rsi"],color='g',width=1),
        # mpl.make_addplot(df["26_rsi"],color='r',width=1),
        # mpl.make_addplot(df["52_rsi"],color='y',width=1),
       #]


        mpl.plot(df, type="candle", style="yahoo", axisoff=True,
                            savefig=address, tight_layout=True,figsize =(4,3))
        
        plt.close("all")
        del df
        del date_list
        del rrng
        del i
        del right_now
        gc.collect()



        index = index + 1
        if index % 10 == 0:
            print(f"{index}/{all_n}", end=" ")
        if index % 160 == 0:
          print(" ")

    print("")
    del dfs
    del suggs
    del index
    del all_n
    gc.collect()



def start(how_many_future_candles, how_many_past_candles, each_row_past):
    folder_name = extract_data(
        how_many_future_candles, how_many_past_candles, each_row_past)
    return folder_name


In [3]:
symbols = get_crypto_syms()
print(f"Symbols : {len(symbols)}")
#symbols = ["btc-usd","eth-usd","trx-usd","ltc-usd","xrp-usd","bnb-usd","FTT-USD"]
symbols = ["btc-usd"]
download_data(symbols,"100d","1h")

Symbols : 1500
 


In [4]:
how_many_future_candles = 5
how_many_past_candles = 1
each_row_past = 60

global which_future_or_past
which_future_or_past = None
if how_many_future_candles > how_many_past_candles:
    which_future_or_past = how_many_future_candles
else:
    which_future_or_past = how_many_past_candles

folder_name = start(how_many_future_candles,how_many_past_candles,each_row_past)
len(os.listdir(f"/content/extracted/{folder_name}"))

Files In Data : 1
Processing File:
File Number 1: 10/2319 20/2319 30/2319 40/2319 50/2319 60/2319 70/2319 80/2319 90/2319 100/2319 110/2319 120/2319 130/2319 140/2319 150/2319 160/2319  
170/2319 180/2319 190/2319 200/2319 210/2319 220/2319 230/2319 240/2319 250/2319 260/2319 270/2319 280/2319 290/2319 300/2319 310/2319 320/2319  
330/2319 340/2319 350/2319 360/2319 370/2319 380/2319 390/2319 400/2319 410/2319 420/2319 430/2319 440/2319 450/2319 460/2319 470/2319 480/2319  
490/2319 500/2319 510/2319 520/2319 530/2319 540/2319 550/2319 560/2319 570/2319 580/2319 590/2319 600/2319 610/2319 620/2319 630/2319 640/2319  
650/2319 660/2319 670/2319 680/2319 690/2319 700/2319 710/2319 720/2319 730/2319 740/2319 750/2319 760/2319 770/2319 780/2319 790/2319 800/2319  
810/2319 820/2319 830/2319 840/2319 850/2319 860/2319 870/2319 880/2319 890/2319 900/2319 910/2319 920/2319 930/2319 940/2319 950/2319 960/2319  
970/2319 980/2319 990/2319 1000/2319 1010/2319 1020/2319 1030/2319 1040/2319 1050/2

2319

In [5]:
#folder_name = '122227'



label = []
data  = []
files = os.listdir(f"/content/extracted/{folder_name}/")
for i, image_name in enumerate(files):
  if image_name.split(".")[1] == "png":
    image = cv2.imread(f"/content/extracted/{folder_name}"+"/"+image_name)
    dim = (256, 128)
    resized = cv2.resize(image, dim)
    data.append(np.array(resized))
    sugg = image_name.split("_")[1].split(".")[0]
    label.append(int(sugg))
data = np.array(data)
data = data / 255
print(data.shape)
xTrain , xTest , yTrain , yTest = train_test_split(data,label,test_size=0.2,random_state=99)

data = None
label = None
print(f"xTrain : {len(xTrain)} \\ xTest : {len(xTest)}")

nytrain = []
nytest = []
yn = 0
nn = 0

for i in yTrain:
  if i == 1:
    nytrain.append([1,0])
    yn += 1
  else:
    nytrain.append([0,1])
    nn += 1
for i in yTest:
  if i == 1:
    nytest.append([1,0])
    yn += 1
  else:
    nytest.append([0,1])
    nn += 1

yTrain = np.array(nytrain)
yTest = np.array(nytest)

print(f"yn: {yn} nn: {nn}")

(2319, 128, 256, 3)
xTrain : 1855 \ xTest : 464
yn: 1119 nn: 1200


In [6]:
model = Sequential()

model.add(Conv2D(128,    (3, 3),activation="relu", input_shape=(xTrain.shape[1], xTrain.shape[2],3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64,    (2, 2),activation="relu",))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(32,    (2, 2),activation="relu",))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(1024,activation="relu"))
model.add(Dense(1024,activation="relu"))
model.add(Dense(750,activation="relu"))
model.add(Dense(512,activation="relu"))
model.add(Dense(2,activation="sigmoid"))

adamax = tf.keras.optimizers.Adamax(
    learning_rate=0.001)
model.compile(loss='categorical_crossentropy',
              optimizer=adamax,
              metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 254, 128)     3584      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 63, 127, 128)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 62, 126, 64)       32832     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 31, 63, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 30, 62, 32)        8224      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 15, 31, 32)       0

In [ ]:
filepath = "/content/checkpoints/{epoch:02d}-{val_accuracy:.2f}.h5"
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=filepath,
    save_weights_only=False,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

#model.fit(xTrain,yTrain,batch_size=64,epochs=30,validation_data=(xTest,yTest), callbacks=model_checkpoint_callback)
model.fit(xTrain,yTrain,batch_size=8,epochs=20,validation_data=(xTest,yTest))


Epoch 1/20
232/232 [==============================] - 176s 754ms/step - loss: 0.6980 - accuracy: 0.5164 - val_loss: 0.6924 - val_accuracy: 0.5431
Epoch 2/20
232/232 [==============================] - 174s 750ms/step - loss: 0.6934 - accuracy: 0.5111 - val_loss: 0.6925 - val_accuracy: 0.5797
Epoch 3/20
232/232 [==============================] - 174s 752ms/step - loss: 0.6868 - accuracy: 0.5666 - val_loss: 0.6765 - val_accuracy: 0.5841
Epoch 4/20
232/232 [==============================] - 174s 752ms/step - loss: 0.5911 - accuracy: 0.6960 - val_loss: 0.6465 - val_accuracy: 0.6293
Epoch 5/20
232/232 [==============================] - 175s 753ms/step - loss: 0.3561 - accuracy: 0.8496 - val_loss: 0.5812 - val_accuracy: 0.7241
Epoch 6/20
135/232 [================>.............] - ETA: 1:09 - loss: 0.1087 - accuracy: 0.9657